In [1]:
import numpy as np
import bqplot.pyplot as plt
from bqplot import CATEGORY10, ColorScale , DateScale, LinearScale, Lines, Axis, Figure , OrdinalScale , Bars , Axis
from lib.ENTSOE import fetch_production
from lib.ENTSOE import fetch_price
import pandas as pd
from datetime import datetime, timedelta
import lib
from pkgutil import iter_modules
import importlib
import os
from ipywidgets import TwoByTwoLayout , Button, Layout, jslink, IntText, IntSlider, FloatSlider, Dropdown
import json
import random
import requests
from ipyleaflet import Map, GeoJSON, WidgetControl, FullScreenControl, ZoomControl
import ipywidgets as widgets
from ipyflex import FlexLayout


with open("./eu-countries.geo.json") as f:
    countries = json.load(f)
    
m = Map(zoom=3,center=(48.30, 23.23), zoom_control=False, layout=dict(height='100%', width='100%'))
m.add_control(ZoomControl(position='topright'))
m.add_control(FullScreenControl())
geo = GeoJSON(data=countries,style={"fillColor": "white", "weight": 0.5},hover_style={"fillColor": "#1f77b4"},name="Countries")
m.add_layer(geo)


data_production = fetch_production('FR')
data_price = fetch_price('FR')
param = []
labels = []


# Populate lists
productions = ["wind","coal","gas","nuclear","biomass","hydro","oil","solar","unknown"]
for production in productions:
    if production in data_production[0]['production'] and data_production[0]['production'][production] is not None:
        param.append([subdata['production'][production] for subdata in data_production])
        labels.append(production)

    
param_production = [param['datetime'] for param in data_production]
time_data = np.array(param_production,dtype='datetime64')
fig_production = plt.figure(title="Italia", animation_duration=1000)
axes_options_1 = {"x": {"label": "time "}, "y": {"label": "production (MWh) ","label_offset" : "6ex"}}
bar1 =  plt.bar(time_data,param,colors=CATEGORY10,labels=labels,display_legend=True)
axes = plt.axes(options = axes_options_1)


param_price = [param['price'] for param in data_price]
param_datetime = [param['datetime'] for param in data_price]
datetime_data = np.array(param_datetime,dtype='datetime64')
fig_price = plt.figure(title="Electricity price")
axes_options_2 = {"x": {"label": "time "}, "y": {"label": "price (€/MWh) ","label_offset" : "6ex"}}
bar2 = plt.bar(datetime_data, param_price)
axes2 = plt.axes(options = axes_options_2)


def update_figure_production(param, labels , time_data , country):
    bar1.y = param
    bar1.x = time_data
    fig_production.title = country
    
def update_figure_price(param_price, datetime_data , country):
    bar2.y = param_price
    bar2.x = datetime_data
    fig_price.title = country

def on_click(event, feature, **kwargs):
  
    data_production1 = fetch_production(feature['properties']['iso_a2'])
    data_price = fetch_price(feature['properties']['iso_a2'])
    param = []
    labels = []
    productions = ["wind","coal","gas","nuclear","biomass","hydro","oil","solar","unknown"]
    for production in productions:
        if production in data_production1[0]['production'] and data_production1[0]['production'][production] is not None:
            param.append([subdata['production'][production] for subdata in data_production1])
            labels.append(production)
    
    param_price = [param1['price'] for param1 in data_price]
    param_datetime = [param['datetime'] for param in data_price]
    update_figure_production(param, labels , time_data , feature['properties']['name'])
    update_figure_price(param_price, datetime_data , feature['properties']['name'])

    
geo.on_click(on_click)



/tmp/ipykernel_31258/1448717279.py:46: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  time_data = np.array(param_production,dtype='datetime64')
/tmp/ipykernel_31258/1448717279.py:55: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  datetime_data = np.array(param_datetime,dtype='datetime64')


In [10]:
w = {'Map':m,'fig_production':fig_production , 'fig_price':fig_price}
header = dict(title='Electricity Production', buttons=[])
FlexLayout(w, style = {'height':'calc(100vh - 60px)'}, header=header, template="Electricity_map.json", editable=False)

FlexLayout(children={'Map': Map(bottom=880.0, center=[47.989921667414194, 23.73046875], controls=(AttributionC…